In [75]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import os 
from datetime import datetime 

In [94]:
df = pd.read_feather('../../../jmarchezi/research-projects/solar-wind-data-with-gaps/outputData/combined_ace_data_resampled.feather')

In [95]:
pd.to_datetime(df['ACEepoch'], format='%Y-%m-%d %H:%M:%S')
df.set_index('ACEepoch', inplace=True, drop=True)
df.index = pd.to_datetime(df.index)

df.dropna(inplace=True)
df.drop(['YR', 'Month', 'Day', 'HR'], axis=1, inplace=True)

In [101]:
time_diff = df.index.to_series().diff()
mask = (time_diff.shift(-1) == pd.Timedelta(minutes=1)) & (time_diff == pd.Timedelta(minutes=1))

print(mask.sum())

# Find consecutive True values in the mask
consecutive_true = mask.cumsum()
mask = mask & (consecutive_true % 2 == 1)

print(mask.sum())

num_true = mask.sum()
max_samples = int(len(df) * 0.2)  # 20% of the total number of rows

# If the number of True values exceeds the maximum samples, randomly sample from the True indices
if num_true > max_samples:
    sample_indices = mask[mask].sample(n=max_samples, random_state=42).index
    mask = mask.index.isin(sample_indices)

samples_to_nan = df[mask]

6278314
3139157


In [102]:
test_vx_nans = df['Vx']
test_vx_nans.loc[test_vx_nans.index.isin(samples_to_nan.index)] = np.nan
print(test_vx_nans)
print(test_vx_nans.isnull().sum())
print(len(df))


ACEepoch
1998-02-05 00:01:00    325.715881
1998-02-05 00:02:00           NaN
1998-02-05 00:03:00    325.696991
1998-02-05 00:04:00           NaN
1998-02-05 00:05:00    324.898804
                          ...    
2020-12-31 23:54:00    362.555084
2020-12-31 23:55:00    359.486938
2020-12-31 23:56:00           NaN
2020-12-31 23:57:00    360.278900
2020-12-31 23:58:00    361.805573
Name: Vx, Length: 6994795, dtype: float64
1410615
6994795


In [103]:
# Check for consecutive rows with NaN values
consecutive_nan = test_vx_nans.isnull() & test_vx_nans.shift().isnull()

# Check if any consecutive NaN rows exist
if consecutive_nan.any():
    print("Consecutive NaN rows exist.")
else:
    print("No consecutive NaN rows.")

No consecutive NaN rows.
